In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('tweet1.csv')

In [3]:
train_df = pd.read_csv('train_data.csv')

In [4]:
test_df = pd.read_csv('test_data.csv')

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import re

In [6]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['tweet'].values.astype(str))
X = tokenizer.texts_to_sequences(df['tweet'].values.astype(str))
X = pad_sequences(X)

In [7]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 42, 128)           256000    
                                                                 
 spatial_dropout1d (Spatial  (None, 42, 128)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 3)                 591       
                                                                 
Total params: 511391 (1.95 MB)
Trainable params: 511391 (1.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [8]:
Y = pd.get_dummies(df['decision']).values

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

In [10]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 2, batch_size=batch_size)

Epoch 1/2
5483/5483 [==============================] - 1443s 263ms/step - loss: 0.3977 - accuracy: 0.8672
Epoch 2/2
5483/5483 [==============================] - 1611s 294ms/step - loss: 0.3264 - accuracy: 0.8959


In [11]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')

1371/1371 [==============================] - 86s 62ms/step - loss: 0.3162 - accuracy: 0.9005
Loss: 0.3162, Accuracy: 0.9005


In [12]:
from sklearn.metrics import classification_report

In [13]:
y_pred_prob = model.predict(X_test)


1371/1371 [==============================] - 81s 59ms/step


ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass targets

In [19]:
Y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels

array([[3.2992646e-02, 9.4632351e-01, 2.0683812e-02],
       [1.1337916e-01, 7.3287839e-01, 1.5374251e-01],
       [1.0620408e-01, 8.1234616e-01, 8.1449859e-02],
       ...,
       [1.2344323e-02, 5.8772939e-04, 9.8706800e-01],
       [1.0494576e-01, 8.0829394e-01, 8.6760297e-02],
       [8.7330248e-03, 1.2386298e-03, 9.9002832e-01]], dtype=float32)

In [21]:
Y_pred = pd.get_dummies(Y_pred).values

In [23]:
# Print classification report
report = classification_report(Y_test, Y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.91      0.71      0.79      8038
           1       0.84      0.99      0.91     13637
           2       0.95      0.92      0.93     22184

   micro avg       0.90      0.90      0.90     43859
   macro avg       0.90      0.87      0.88     43859
weighted avg       0.90      0.90      0.90     43859
 samples avg       0.90      0.90      0.90     43859



In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [26]:
precision = precision_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
print(precision, f1, recall)

0.9049756954616751 0.8983887567304977 0.9004765270526004
